## Ocean Surface metrics: Long-term Changes

In this notebook, we examine long-term changes (over 10-year period) in the overturning strength in the subpolar North Atlantic. Correspondingly, changes in SST, surface heat flux and sea surface height are analysed at high latitudes.

We focus on winter season (DJF) and simulation started on 1 Nov 2008. The approach is then extended to all simulations (see relevant python script).